In [9]:
import os
import re
import sqlite3
from datetime import datetime, timedelta
from typing import Dict, List, Optional, Any, Tuple

from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

# Load environment variables
load_dotenv()

# Initialize the Gemini model
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", 
                             google_api_key=os.environ["GEMINI_API_KEY"],
                             temperature=0.1)

# Define a prompt template for the appointment assistant
appointment_prompt_template = """
You are an AI Appointment Booking Assistant designed to help users schedule and manage their appointments.

CONTEXT:
The user is interacting with you through a chat interface. Your primary functions are:
1. Helping users book new appointments
2. Retrieving information about existing appointments

CURRENT STATE: {current_state}
USER INFORMATION: {user_info}

USER INPUT: {user_input}

Your task is to respond helpfully and naturally while guiding the user through the appointment booking or retrieval process.
If the user provides unclear or incomplete information, politely ask for clarification.

FORMATTING GUIDELINES:
- Present appointment details in a clean, organized format using clear headings and sections.
- Use bullet points or numbered lists where appropriate.
- For appointment confirmations, create a visually distinct "Appointment Confirmation" section.
- For appointment listings, format each appointment in a way that's easy to scan quickly.
- Add emoji for relevant concepts (📅 for dates, ⏰ for time, 📝 for purpose, etc.)

Keep your responses concise, friendly, and focused on the appointment booking task.

RESPONSE:
"""

# Create the prompt template object
prompt = PromptTemplate(
    input_variables=["current_state", "user_info", "user_input"],
    template=appointment_prompt_template
)

# Create the LLM chain
appointment_chain = LLMChain(llm=llm, prompt=prompt)

# Database setup
def setup_database():
    conn = sqlite3.connect('appointments.db')
    cursor = conn.cursor()
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS appointments (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        name TEXT NOT NULL,
        email TEXT NOT NULL,
        date TEXT NOT NULL,
        time TEXT NOT NULL,
        purpose TEXT NOT NULL,
        timestamp TEXT NOT NULL
    )
    ''')
    conn.commit()
    conn.close()

# Check if an appointment already exists
def is_duplicate_appointment(email: str, date: str, time: str) -> bool:
    conn = sqlite3.connect('appointments.db')
    cursor = conn.cursor()
    cursor.execute(
        "SELECT COUNT(*) FROM appointments WHERE email = ? AND date = ? AND time = ?",
        (email, date, time)
    )
    count = cursor.fetchone()[0]
    conn.close()
    return count > 0

# Store appointment in database
def store_appointment(name: str, email: str, date: str, time: str, purpose: str) -> Tuple[bool, str]:
    try:
        # Validate all required fields
        if not all([name, email, date, time, purpose]):
            return False, "Missing required information"
        
        # Check for duplicate appointment
        if is_duplicate_appointment(email, date, time):
            return False, "duplicate"
        
        conn = sqlite3.connect('appointments.db')
        cursor = conn.cursor()
        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        cursor.execute(
            "INSERT INTO appointments (name, email, date, time, purpose, timestamp) VALUES (?, ?, ?, ?, ?, ?)",
            (name, email, date, time, purpose, timestamp)
        )
        conn.commit()
        conn.close()
        return True, "success"
    except Exception as e:
        print(f"Error storing appointment: {e}")
        return False, str(e)

# Retrieve appointments by email
def get_appointments_by_email(email: str) -> List[Dict]:
    conn = sqlite3.connect('appointments.db')
    cursor = conn.cursor()
    cursor.execute("SELECT name, email, date, time, purpose FROM appointments WHERE email = ?", (email,))
    appointments = cursor.fetchall()
    conn.close()
    
    result = []
    for app in appointments:
        result.append({
            "name": app[0],
            "email": app[1],
            "date": app[2],
            "time": app[3],
            "purpose": app[4]
        })
    
    return result

# Extract email from user input
def extract_email(text: str) -> str:
    email_match = re.search(r"[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}", text)
    if email_match:
        return email_match.group(0)
    return ""

# Check if input contains booking intent
def has_booking_intent(text: str) -> bool:
    booking_indicators = [
        "book", "schedule", "make", "set", "create", "arrange", "new appointment", 
        "fix", "plan", "reserve", "want appointment", "need appointment", "1", "option 1",
        "first option", "booking"
    ]
    return any(indicator.lower() in text.lower() for indicator in booking_indicators)

# Check if input contains checking/retrieval intent
def has_checking_intent(text: str) -> bool:
    checking_indicators = [
        "check", "get", "show", "list", "find", "view", "see", "retrieve", "tell", 
        "what", "when", "my appointment", "appointment detail", "status", "2", "option 2",
        "second option", "checking"
    ]
    return any(indicator.lower() in text.lower() for indicator in checking_indicators)

# Validate date format
def is_valid_date(date_str: str) -> bool:
    # Check common date formats: DD/MM/YYYY, MM/DD/YYYY, YYYY-MM-DD
    date_patterns = [
        re.compile(r"^(\d{1,2})/(\d{1,2})/(\d{4})$"),  # DD/MM/YYYY or MM/DD/YYYY
        re.compile(r"^(\d{4})-(\d{1,2})-(\d{1,2})$"),  # YYYY-MM-DD
        re.compile(r"^(\d{1,2})-(\d{1,2})-(\d{4})$")   # DD-MM-YYYY or MM-DD-YYYY
    ]
    
    # Check if date matches any of the patterns
    if not any(pattern.match(date_str) for pattern in date_patterns):
        return False
    
    # Try to parse the date
    try:
        # First try with common formats
        formats = ["%d/%m/%Y", "%m/%d/%Y", "%Y-%m-%d", "%d-%m-%Y", "%m-%d-%Y"]
        for fmt in formats:
            try:
                parsed_date = datetime.strptime(date_str, fmt)
                # Check if date is in the future (or today)
                if parsed_date.date() >= datetime.now().date():
                    return True
            except ValueError:
                continue
        return False
    except Exception:
        return False

# Validate time format
def is_valid_time(time_str: str) -> bool:
    # Check multiple time formats: 14:30, 2:30 PM, 2PM, etc.
    time_patterns = [
        re.compile(r"^([01]?[0-9]|2[0-3]):([0-5][0-9])$"),  # 24-hour format like 14:30
        re.compile(r"^(1[0-2]|0?[1-9]):([0-5][0-9])\s?(AM|PM|am|pm)$"),  # 12-hour with minutes
        re.compile(r"^(1[0-2]|0?[1-9])\s?(AM|PM|am|pm)$")  # 12-hour without minutes
    ]
    
    return any(pattern.match(time_str) for pattern in time_patterns)

# Standardize time format
def standardize_time(time_str: str) -> str:
    # If already in 24-hour format like 14:30
    if re.match(r"^([01]?[0-9]|2[0-3]):([0-5][0-9])$", time_str):
        return time_str
    
    # If in 12-hour format with minutes like 2:30 PM
    match = re.match(r"^(1[0-2]|0?[1-9]):([0-5][0-9])\s?(AM|PM|am|pm)$", time_str)
    if match:
        hour, minute, period = match.groups()
        hour = int(hour)
        if period.lower() == "pm" and hour < 12:
            hour += 12
        elif period.lower() == "am" and hour == 12:
            hour = 0
        return f"{hour:02d}:{minute}"
    
    # If in 12-hour format without minutes like 2PM
    match = re.match(r"^(1[0-2]|0?[1-9])\s?(AM|PM|am|pm)$", time_str)
    if match:
        hour, period = match.groups()
        hour = int(hour)
        if period.lower() == "pm" and hour < 12:
            hour += 12
        elif period.lower() == "am" and hour == 12:
            hour = 0
        return f"{hour:02d}:00"
    
    return time_str  # Return original if no pattern matches

# For more complex responses, use the LLM
def get_llm_response(state, user_input, booking_info=None):
    # Prepare user info summary
    if booking_info:
        user_info = ", ".join([f"{k}: {v}" for k, v in booking_info.items() if v])
    else:
        user_info = "No booking information available yet"
    
    # Use the LLM chain to generate a response
    response = appointment_chain.run(
        current_state=state,
        user_info=user_info,
        user_input=user_input
    )
    
    return response.strip()

# Format appointment details in a clear, readable way
def format_appointment_details(booking_info):
    return f"""
📋 APPOINTMENT DETAILS 📋
---------------------------
👤 Name: {booking_info["name"]}
📧 Email: {booking_info["email"]}
📅 Date: {booking_info["date"]}
⏰ Time: {booking_info["time"]}
📝 Purpose: {booking_info["purpose"]}
---------------------------
"""

# Validate booking information
def validate_booking_info(booking_info):
    missing_fields = []
    invalid_fields = []
    
    # Check required fields
    for field in ["name", "email", "date", "time", "purpose"]:
        if field not in booking_info or not booking_info[field].strip():
            missing_fields.append(field)
    
    # Validate fields if present
    if "email" in booking_info and booking_info["email"]:
        if not re.match(r"[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}", booking_info["email"]):
            invalid_fields.append("email")
    
    if "date" in booking_info and booking_info["date"]:
        if not is_valid_date(booking_info["date"]):
            invalid_fields.append("date")
    
    if "time" in booking_info and booking_info["time"]:
        if not is_valid_time(booking_info["time"]):
            invalid_fields.append("time")
    
    return missing_fields, invalid_fields

# Main chat function
def chat():
    # Set up the database
    setup_database()
    
    # Welcome message
    welcome_message = """
Hello! I'm your Appointment Booking Assistant. Please choose an option:

1. Book a new appointment
2. Check your existing appointments

Type '1' for booking, '2' for checking, or 'exit' to quit.
    """
    print("Assistant:", welcome_message)
    
    state = "choosing"  # Initial state
    booking_info = {}  # Store booking information
    
    while True:
        user_input = input("You: ")
        
        if user_input.lower() in ["exit", "quit", "bye"]:
            # Use LLM for goodbye message
            farewell = get_llm_response("farewell", user_input)
            print(f"Assistant: {farewell}")
            break
            
        # For unclear inputs or more complex cases, use the LLM
        if user_input.strip() and len(user_input.split()) > 3:
            # Check if we should use rule-based logic or LLM
            use_llm = not (
                (state == "choosing" and (has_booking_intent(user_input) or has_checking_intent(user_input))) or
                (state == "booking_email" and extract_email(user_input)) or
                (state in ["booking_name", "booking_date", "booking_time", "booking_purpose"] and user_input.strip())
            )
            
            if use_llm:
                response = get_llm_response(state, user_input, booking_info)
                print(f"Assistant: {response}")
                # Check if we should still process with rule-based logic
                if "I'll help you book" in response and state == "choosing":
                    state = "booking_name"
                elif "I'll help you check" in response and state == "choosing":
                    state = "checking_email"
                continue
            
        # State machine to handle the conversation flow
        if state == "choosing":
            if has_booking_intent(user_input):
                state = "booking_name"
                print("Assistant: Let's book your appointment. What's your name?")
            elif has_checking_intent(user_input):
                state = "checking_email"
                print("Assistant: I'll help you check your appointments. Please provide your email address.")
            else:
                # Use LLM for unclear intent
                response = get_llm_response("unclear_intent", user_input)
                print(f"Assistant: {response}")
                
        elif state == "booking_name":
            if user_input.strip():
                booking_info["name"] = user_input.strip()
                state = "booking_email"
                print("Assistant: Great! Now, what's your email address?")
            else:
                print("Assistant: I need your name to proceed. Please provide your name.")
            
        elif state == "booking_email":
            email = extract_email(user_input)
            if email:
                booking_info["email"] = email
                state = "booking_date"
                print("Assistant: Perfect! For which date would you like to book the appointment? (e.g., 25/04/2025)")
            else:
                print("Assistant: I couldn't detect a valid email address. Please provide a valid email (e.g., example@domain.com).")
                
        elif state == "booking_date":
            date_input = user_input.strip()
            if is_valid_date(date_input):
                booking_info["date"] = date_input
                state = "booking_time"
                print("Assistant: What time would you prefer for your appointment? (e.g., 14:30 or 2:30 PM)")
            else:
                print("Assistant: Please provide a valid date format (e.g., 25/04/2025, 2025-04-25). Make sure the date is not in the past.")
            
        elif state == "booking_time":
            time_input = user_input.strip()
            if is_valid_time(time_input):
                # Standardize time format for storage
                booking_info["time"] = standardize_time(time_input)
                state = "booking_purpose"
                print("Assistant: What's the purpose of your appointment?")
            else:
                print("Assistant: Please provide a valid time format (e.g., 14:30, 2:30 PM, or 2PM).")
            
        elif state == "booking_purpose":
            if user_input.strip():
                booking_info["purpose"] = user_input.strip()
                
                # Final validation of all booking info
                missing_fields, invalid_fields = validate_booking_info(booking_info)
                
                if missing_fields:
                    missing_str = ", ".join(missing_fields)
                    print(f"Assistant: There are missing fields in your booking: {missing_str}. Let's fill these in.")
                    
                    # Return to the first missing field
                    state = f"booking_{missing_fields[0]}"
                    print(f"Assistant: Please provide your {missing_fields[0]}:")
                    continue
                
                if invalid_fields:
                    invalid_str = ", ".join(invalid_fields)
                    print(f"Assistant: There are invalid fields in your booking: {invalid_str}. Let's correct these.")
                    
                    # Return to the first invalid field
                    state = f"booking_{invalid_fields[0]}"
                    print(f"Assistant: Please provide a valid {invalid_fields[0]}:")
                    continue
                
                # Store appointment in database
                success, message = store_appointment(
                    booking_info["name"],
                    booking_info["email"],
                    booking_info["date"],
                    booking_info["time"],
                    booking_info["purpose"]
                )
                
                if success:
                    # Format appointment details
                    details = format_appointment_details(booking_info)
                    print(f"Assistant: Your appointment has been successfully booked! 🎉\n{details}")
                    
                    # Add options for next steps
                    print("""
What would you like to do next?
1. Book another appointment
2. Check your appointments
Type 'exit' to quit.
                    """)
                elif message == "duplicate":
                    print(f"Assistant: It appears you already have an appointment scheduled for {booking_info['date']} at {booking_info['time']}.")
                    print("Would you like to:")
                    print("1. Choose a different date or time")
                    print("2. Check your existing appointments")
                    print("3. Start over with a new booking")
                    
                    duplicate_choice = input("You: ")
                    
                    if "1" in duplicate_choice or "different" in duplicate_choice.lower():
                        state = "booking_date"
                        print("Assistant: Let's choose a different date. For which date would you like to book the appointment? (e.g., 25/04/2025)")
                        continue
                    elif "2" in duplicate_choice or "check" in duplicate_choice.lower():
                        appointments = get_appointments_by_email(booking_info["email"])
                        print(f"Assistant: Here are your existing appointments:")
                        for i, app in enumerate(appointments, 1):
                            details = f"""
📋 APPOINTMENT #{i} 📋
---------------------------
👤 Name: {app['name']}
📧 Email: {app['email']}
📅 Date: {app['date']}
⏰ Time: {app['time']}
📝 Purpose: {app['purpose']}
---------------------------
"""
                            print(details)
                        state = "choosing"
                    else:
                        booking_info = {}
                        state = "choosing"
                        print("""
What would you like to do next?
1. Book a new appointment
2. Check your appointments
Type 'exit' to quit.
                        """)
                else:
                    # Use LLM for error message
                    error_msg = get_llm_response("booking_error", message, booking_info)
                    print(f"Assistant: {error_msg}")
                    state = "choosing"
            else:
                print("Assistant: I need to know the purpose of your appointment. Please briefly describe why you're scheduling this appointment.")
                continue
            
            # Reset for next operation
            booking_info = {}
            state = "choosing"
            
        elif state == "checking_email":
            email = extract_email(user_input)
            if email:
                appointments = get_appointments_by_email(email)
                
                if appointments:
                    print(f"Assistant: I found {len(appointments)} appointment(s) for {email}:")
                    
                    for i, app in enumerate(appointments, 1):
                        details = f"""
📋 APPOINTMENT #{i} 📋
---------------------------
👤 Name: {app['name']}
📧 Email: {app['email']}
📅 Date: {app['date']}
⏰ Time: {app['time']}
📝 Purpose: {app['purpose']}
---------------------------
"""
                        print(details)
                    
                else:
                    # Use LLM for no appointments found
                    response = get_llm_response("no_appointments", "", {"email": email})
                    print(f"Assistant: {response}")
                
                print("""
What would you like to do next?
1. Book a new appointment
2. Check another email's appointments
Type 'exit' to quit.
                """)
                state = "choosing"
            else:
                print("Assistant: I couldn't detect a valid email address. Please provide a valid email (e.g., example@domain.com).")

if __name__ == "__main__":
    chat()

Assistant: 
Hello! I'm your Appointment Booking Assistant. Please choose an option:

1. Book a new appointment
2. Check your existing appointments

Type '1' for booking, '2' for checking, or 'exit' to quit.
    
Assistant: Okay, goodbye! If you need help with appointments in the future, just let me know.
